In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('../dataset/complete_dataset.csv')

C:\Users\janko\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1,3,5,6,7,8,9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df.shape

(10927181, 19)

In [3]:
df.head()

,filename,text,IMDB_ID,genre,year,production_region,corpus,duration,directors,writers,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
0,6584498.xml,( Rückblick ) Waren Sie neulich mit uns in die...,tt1147427,"Comedy,Family,Romance",NaN,NaN,untokenisiert,NaN,nm0240047,nm0226201,tvEpisode,Príbeh séfova zástupce,Príbeh séfova zástupce,0.0,NaN,NaN,NaN,NaN,NaN
1,6977710.xml,FÜNF FREUNDE FÜR ALLE FÄLLE FÜNF FREUNDE FÜR A...,tt1294864,"Animation,Comedy",NaN,NaN,untokenisiert,NaN,NaN,"nm0509769,nm0537542,nm0875635",tvEpisode,The Case of Allie's Really Very Bad Singing,The Case of Allie's Really Very Bad Singing,0.0,NaN,NaN,22,NaN,NaN
2,6977710.xml,FÜNF FREUNDE FÜR ALLE FÄLLE FÜNF FREUNDE FÜR A...,tt1294864,"Animation,Comedy",NaN,NaN,untokenisiert,NaN,NaN,"nm0509769,nm0537542,nm0875635",tvEpisode,The Case of Allie's Really Very Bad Singing,The Case of Allie's Really Very Bad Singing,0.0,NaN,NaN,22,NaN,NaN
3,6977706.xml,FÜNF FREUNDE FÜR ALLE FÄLLE FÜNF FREUNDE FÜR A...,tt1294865,"Animation,Comedy",NaN,NaN,untokenisiert,NaN,NaN,"nm0537542,nm0875635",tvEpisode,"The Case of the Impolite, Snarly Thing","The Case of the Impolite, Snarly Thing",0.0,NaN,NaN,22,NaN,NaN
4,6979067.xml,FÜNF FREUNDE - FÜR ALLE FÄLLE FÜNF FREUNDE - F...,tt1328700,"Animation,Comedy",NaN,NaN,untokenisiert,NaN,NaN,"nm0247695,nm0537542,nm0875635",tvEpisode,The Case of the Felon with Frosty Fingers,The Case of the Felon with Frosty Fingers,0.0,NaN,NaN,22,NaN,NaN


In [14]:
df.dropna(subset=['text', 'genre', 'year', 'production_region'], inplace=True)
df.shape

(21645, 19)

In [15]:
df

,filename,text,IMDB_ID,genre,year,production_region,corpus,duration,directors,writers,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
109,3653850.xml,[ Subtitles by Subturtles ] Du sollst nach Hau...,tt1175,"Drama,Romance",1912.0,French,untokenisiert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,6581015.xml,HINTER DER LEIWAND Die Anwendung für die Arbei...,tt6414,"Comedy,Romance,Short",1916.0,English,untokenisiert,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,6564964.xml,DER GRAF Maße : Hals 30 cm - Ohr - 4 cm Mounth...,tt6548,"Comedy,Short",1916.0,English,untokenisiert,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,6412883.xml,"Chaplin bei der Feuerwehr "" Lass doch das Haus...",tt6684,"Comedy,Short",1916.0,English,untokenisiert,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,6565009.xml,Der Abenteurer Der Mann Jagd . Ein morgendlich...,tt7613,"Comedy,Short",1917.0,English,untokenisiert,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59654,7069099.xml,""" Wir bauen Dämme in uns auf , um die Betrübni...",tt3731580,"Drama,Mystery",2017.0,English,untokenisiert,92,nm4476940,nm4476940,movie,We Don't Belong Here,We Don't Belong Here,0.0,2017.0,NaN,92,5.6,1260.0
59664,6985504.xml,KONGO Das Überleben der Stärksten . Das ist da...,tt3874544,"Animation,Comedy,Family",2017.0,English,untokenisiert,97,nm0569891,"nm0567112,nm2807682",movie,The Boss Baby,The Boss Baby,0.0,2017.0,NaN,97,6.3,102996.0
59665,7045432.xml,KONGO Das Überleben der stärksten . Das ist da...,tt3874544,"Animation,Comedy,Family",2017.0,English,untokenisiert,97,nm0569891,"nm0567112,nm2807682",movie,The Boss Baby,The Boss Baby,0.0,2017.0,NaN,97,6.3,102996.0
59666,7062268.xml,MISSOURI ERDE 1980 Ein Mädchen in einer Hafens...,tt3896198,"Action,Adventure,Sci-Fi",2017.0,English,untokenisiert,136,nm0348181,"nm0348181,nm3966115,nm5037683,nm1921680,nm3238...",movie,Guardians of the Galaxy Vol. 2,Guardians of the Galaxy Vol. 2,0.0,2017.0,NaN,136,7.6,538702.0


In [16]:
df_sample = df.sample(frac=0.1)
df_sample.shape

(2164, 19)

In [17]:
df_sample.head()

,filename,text,IMDB_ID,genre,year,production_region,corpus,duration,directors,writers,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,averageRating,numVotes
21277,6274689.xml,DIE CASTING COUCH Heiße Dates und sexy Girls A...,tt398839,Comedy,2006.0,English,untokenisiert,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3266,4501607.xml,In Deckung ! Da drüben sind noch mehr ! Zurück...,tt62038,"Crime,Drama,Mystery",1967.0,"English, Italian, Spanish, French",untokenisiert,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25256,3482023.xml,"91 , 92 , es ist hier unten . - Und wenn sie u...",tt1129442,"Action,Adventure,Crime",2008.0,"English, French, Ukrainian, Russian, Hungarian",untokenisiert,104,nm0576298,"nm0000108,nm0436543",movie,Transporter 3,Transporter 3,0.0,2008.0,NaN,104,6.1,153685.0
48098,6481424.xml,"( Regen rauscht , es donnert ) ( Mädchen ) Ich...",tt3334794,"Comedy,Drama",2014.0,German,untokenisiert,104,nm1103518,"nm1103518,nm2058345",movie,About a Girl,About a Girl,0.0,2014.0,NaN,104,6.6,691.0
34416,5080647.xml,Wisst ihr noch ? Arthur wollte seinen Großelte...,tt940656,"Adventure,Animation,Family",2010.0,English,untokenisiert,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
